In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')

In [2]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'config.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [3]:
from rest_framework import serializers
from rest_framework.exceptions import ValidationError,ErrorDetail

In [6]:
class TestErrorsSerializer(serializers.Serializer):
    field1 = serializers.CharField()
    field2 = serializers.CharField(required=True)
    field3 = serializers.CharField(required=True)


# class ChildSer


serializer = TestErrorsSerializer(data={
    'field1': 's',
    # 'field2': 's',
    # 'field3': 's',
})
serializer.is_valid()


errors = serializer.errors

print(f'{errors = }')
print(f'{type(errors) = }')

validation_error = ValidationError(errors)

print(f'{validation_error = }')
print(f'{validation_error.detail = }')

errors = {'field2': [ErrorDetail(string='This field is required.', code='required')], 'field3': [ErrorDetail(string='This field is required.', code='required')]}
type(errors) = <class 'rest_framework.utils.serializer_helpers.ReturnDict'>
validation_error = ValidationError({'field2': [ErrorDetail(string='This field is required.', code='required')], 'field3': [ErrorDetail(string='This field is required.', code='required')]})
validation_error.detail = {'field2': [ErrorDetail(string='This field is required.', code='required')], 'field3': [ErrorDetail(string='This field is required.', code='required')]}


In [7]:
import json

print(f'{json.dumps(errors) = }')

json.dumps(errors) = '{"field2": ["This field is required."], "field3": ["This field is required."]}'


In [8]:
class NestedSerializer(serializers.Serializer):
    nested_field = serializers.IntegerField()

class MySerializer(serializers.Serializer):
    char_field = serializers.CharField(max_length=50)
    nested = NestedSerializer()
    list_field = serializers.ListField(child=serializers.IntegerField())

# Example usage
data = {
    "char_field": "Example String",
    "nested": {"nested_field": "Nested Data"},
    "list_field": [1, 2, 3, 'dasds']
}

serializer = MySerializer(data=data)
print(serializer.is_valid())  # True
print(f'{serializer.validated_data}') 
print(f'{serializer.errors}') 


print(f'{json.dumps(serializer.errors) = }')

False
{}
{'nested': {'nested_field': [ErrorDetail(string='A valid integer is required.', code='invalid')]}, 'list_field': {3: [ErrorDetail(string='A valid integer is required.', code='invalid')]}}
json.dumps(serializer.errors) = '{"nested": {"nested_field": ["A valid integer is required."]}, "list_field": {"3": ["A valid integer is required."]}}'


In [12]:
try:
    print(serializer.is_valid(raise_exception=True))  # True
except Exception as e:
    print(f'{type(e) = }')
    print(f'{e.detail = }')
    print(f'{json.dumps(e.detail) = }')


type(e) = <class 'rest_framework.exceptions.ValidationError'>
e.detail = {'nested': {'nested_field': [ErrorDetail(string='A valid integer is required.', code='invalid')]}, 'list_field': {3: [ErrorDetail(string='A valid integer is required.', code='invalid')]}}
json.dumps(e.detail) = '{"nested": {"nested_field": ["A valid integer is required."]}, "list_field": {"3": ["A valid integer is required."]}}'
